In [4]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [5]:
train_ds = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_ds = train_ds.flow_from_directory('training_set/training_set', target_size=(64, 64), 
                                        batch_size=32, class_mode='binary')

Found 8005 images belonging to 2 classes.


In [6]:
test_ds = ImageDataGenerator(rescale = 1./255)
test_ds = test_ds.flow_from_directory('test_set/test_set/', target_size=(64, 64), batch_size=32, 
                                      class_mode='binary')                                

Found 2023 images belonging to 2 classes.


In [7]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [8]:
model.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [9]:
model.add(tf.keras.layers.Flatten())

In [10]:
model.add(tf.keras.layers.Dense(units=128, activation='relu'))


In [11]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
model.load_weights('checkpoints/model_at_20.h5')

In [14]:
def predict_image(filename, model, i):
    predict = tf.keras.utils.load_img(filename, target_size = (64, 64))   
    predict_modified = tf.keras.utils.img_to_array(predict)
    predict_modified = predict_modified / 255
    predict_modified = np.expand_dims(predict_modified, axis = 0)
    result = model.predict(predict_modified)
    if result[0][0] >= 0.5:
        prediction = 'dog'
        probability = result[0][0]
        print("Prediction = " + prediction)
        print ("probability = " + str(probability))
    else:
        prediction = 'cat'
        probability = 1 - result[0][0]
        print("Prediction = " + prediction)
        print ("probability = " + str(probability))

In [15]:
for i in range(1, 9):
    predict_image(f'cat_dog_to_predict/{i}.jpg', model, i)

1/1 [==============================] - 0s 161ms/step
Prediction = dog
probability = 0.66152406
1/1 [==============================] - 0s 24ms/step
Prediction = dog
probability = 0.892339
1/1 [==============================] - 0s 24ms/step
Prediction = dog
probability = 0.98126984
1/1 [==============================] - 0s 25ms/step
Prediction = dog
probability = 0.99773496
1/1 [==============================] - 0s 23ms/step
Prediction = cat
probability = 0.9069023430347443
1/1 [==============================] - 0s 25ms/step
Prediction = dog
probability = 0.9530731
1/1 [==============================] - 0s 24ms/step
Prediction = cat
probability = 0.9999987125960388
1/1 [==============================] - 0s 28ms/step
Prediction = cat
probability = 0.8632197827100754
